# Préférences et Théorie du Vote

**Navigation**: [← Tweety-8-Agent-Dialogues](Tweety-8-Agent-Dialogues.ipynb) | [Index](Tweety-1-Setup.ipynb)

---

## Objectifs pédagogiques

1. Comprendre la représentation des ordres de préférence
2. Découvrir les méthodes d'agrégation de préférences (théorie du choix social)
3. Explorer les règles de vote classiques (Borda, Condorcet, Copeland)
4. Comprendre le lien entre préférences et argumentation

## Prérequis

Exécutez d'abord [Tweety-1-Setup.ipynb](Tweety-1-Setup.ipynb) pour configurer l'environnement JVM.

## Module TweetyProject couvert

- `org.tweetyproject.preferences` - Ordres de préférence et agrégation

In [1]:
# --- Initialisation JVM Tweety + Outils Externes ---
print("--- Verification JVM Tweety + Outils ---")
jvm_ready = False

import jpype
import jpype.imports
import os
import pathlib
import shutil
import platform

# === Configuration COMPLETE des outils externes ===
EXTERNAL_TOOLS = {
    "CLINGO": "",
    "SPASS": "",
    "EPROVER": "",
    "SAT_SOLVER_PYTHON": "",
    "MARCO": "",
}

def get_tool_path(tool_name):
    """Retourne le chemin valide d'un outil ou None."""
    path_str = EXTERNAL_TOOLS.get(tool_name, "")
    if not path_str:
        return None
    if shutil.which(path_str):
        return path_str
    path_obj = pathlib.Path(path_str)
    if path_obj.is_file():
        return str(path_obj.resolve())
    if path_obj.is_dir():
        return str(path_obj.resolve())
    return None

# --- Auto-detection des outils ---
system = platform.system()
exe_suffix = ".exe" if system == "Windows" else ""

# 1. Clingo (ASP solver) - Tweety attend le REPERTOIRE
for cp in [shutil.which("clingo"), pathlib.Path(f"ext_tools/clingo/clingo{exe_suffix}"),
           pathlib.Path(f"../ext_tools/clingo/clingo{exe_suffix}")]:
    if cp and (isinstance(cp, str) or cp.exists()):
        parent = pathlib.Path(cp).parent if isinstance(cp, str) else cp.parent
        EXTERNAL_TOOLS["CLINGO"] = str(parent.resolve())
        break

# 2. SPASS (Modal logic prover)
for sp in [shutil.which("SPASS"), pathlib.Path(f"ext_tools/spass/SPASS{exe_suffix}"),
           pathlib.Path(f"../ext_tools/spass/SPASS{exe_suffix}")]:
    if sp and (isinstance(sp, str) or sp.exists()):
        EXTERNAL_TOOLS["SPASS"] = str(pathlib.Path(sp).resolve()) if isinstance(sp, pathlib.Path) else sp
        break

# 3. EProver (FOL theorem prover)
for ep in [shutil.which("eprover"), pathlib.Path(f"../ext_tools/EProver/eprover{exe_suffix}"),
           pathlib.Path(f"ext_tools/EProver/eprover{exe_suffix}")]:
    if ep:
        ep_path = pathlib.Path(ep) if isinstance(ep, str) else ep
        if ep_path.exists():
            EXTERNAL_TOOLS["EPROVER"] = str(ep_path.resolve())
            break

# 4. SAT Solver Python (CaDiCaL, Glucose via pySAT)
for sat in [pathlib.Path("../ext_tools/sat_solver.py"), pathlib.Path("ext_tools/sat_solver.py")]:
    if sat.exists():
        EXTERNAL_TOOLS["SAT_SOLVER_PYTHON"] = str(sat.resolve())
        break

# 5. MARCO (MUS enumerator avec Z3)
for mp in [pathlib.Path("../ext_tools/marco.py"), pathlib.Path("ext_tools/marco.py")]:
    if mp.exists():
        EXTERNAL_TOOLS["MARCO"] = str(mp.resolve())
        break

# === Initialisation JVM ===
if jpype.isJVMStarted():
    print("JVM deja en cours d'execution.")
    jvm_ready = True
else:
    jdk_portable = None
    for jdk_path in [pathlib.Path("jdk-17-portable"), pathlib.Path("../Argument_Analysis/jdk-17-portable")]:
        if jdk_path.exists():
            zulu_dirs = list(jdk_path.glob("zulu*"))
            if zulu_dirs:
                jdk_portable = zulu_dirs[0]
                os.environ["JAVA_HOME"] = str(jdk_portable.resolve())
                print(f"JDK portable: {jdk_portable.name}")
                break

    if not os.environ.get("JAVA_HOME"):
        print("ERREUR: JAVA_HOME non defini et JDK portable non trouve.")
    else:
        LIB_DIR = pathlib.Path("libs")
        if not LIB_DIR.exists():
            LIB_DIR = pathlib.Path("../Argument_Analysis/libs")

        if LIB_DIR.exists():
            jar_files = list(LIB_DIR.glob("*.jar"))
            if jar_files:
                classpath = os.pathsep.join(str(j.resolve()) for j in jar_files)
                try:
                    jpype.startJVM(classpath=[classpath])
                    print(f"JVM demarree avec {len(jar_files)} JARs.")
                    jvm_ready = True
                except Exception as e:
                    print(f"Erreur demarrage JVM: {e}")

# === Resume des outils ===
if jvm_ready:
    print("\n--- Outils disponibles ---")
    for tool, path in EXTERNAL_TOOLS.items():
        if path:
            short_path = path.split(os.sep)[-1] if len(path) > 30 else path
            print(f"  {tool}: {short_path}")
    print(f"\nJVM prete. Outils: {sum(1 for t,p in EXTERNAL_TOOLS.items() if p)}/{len(EXTERNAL_TOOLS)}")

--- Verification JVM Tweety + Outils ---
JDK portable: zulu17.50.19-ca-jdk17.0.11-win_x64
JVM demarree avec 35 JARs.

--- Outils disponibles ---
  CLINGO: clingo
  SPASS: SPASS.exe
  EPROVER: eprover.exe
  SAT_SOLVER_PYTHON: sat_solver.py
  MARCO: marco.py

JVM prete. Outils: 5/5


## Partie 7 : Préférences et Agrégation

La théorie des préférences est fondamentale en choix social, économie et argumentation. TweetyProject fournit des outils pour représenter et agréger les préférences individuelles en préférences collectives.

### 7.1 Ordres de Préférence

Un **ordre de préférence** est une relation binaire sur un ensemble d'alternatives, généralement:
- **Totale**: Toutes les paires sont comparables
- **Transitive**: Si A > B et B > C, alors A > C
- **Antisymétrique**: Si A > B, alors non B > A

TweetyProject représente les préférences via:
- `PreferenceOrder<T>`: Ordre de préférence sur des éléments de type T
- Méthodes de comparaison et manipulation des ordres

In [2]:
# --- 7.1 Ordres de Préférence ---
print("\n--- 7.1 Ordres de Preference ---")

if not jvm_ready:
    print("ERREUR: JVM non demarree.")
else:
    print("JVM prete. Exploration du module preferences...")
    prefs_imports_ok = False
    try:
        import jpype
        from jpype.types import *
        
        # Exploration du package preferences
        print("\n--- Exploration du package org.tweetyproject.preferences ---")
        
        # Classes connues dans le module preferences
        known_classes = [
            "PreferenceOrder",
            "PreferenceOrderViewer",
            "Ranking",
            "ranking.Functions",
            "ranking.LevelingFunction", 
            "io.PreferenceParser",
            "update.Update",
            "aggregation.BordaRule",
            "aggregation.CopelandRule",
        ]
        
        print("Classes du module preferences:")
        found_classes = []
        for cls_name in known_classes:
            try:
                full_name = f"org.tweetyproject.preferences.{cls_name}"
                cls = jpype.JClass(full_name)
                print(f"   [OK] {cls_name}")
                found_classes.append(cls_name)
            except Exception:
                print(f"   [--] {cls_name} (non trouve)")
        
        if found_classes:
            prefs_imports_ok = True
            print(f"\n{len(found_classes)} classes trouvees.")
        else:
            print("\nAucune classe preferences trouvee - le JAR peut etre manquant.")

        # --- Exemple conceptuel de préférences ---
        if prefs_imports_ok:
            print("\n--- Exemple Conceptuel: Election ---")
            print("""
Scenario: 3 votants, 4 candidats (A, B, C, D)

Preferences des votants:
- Votant 1: A > B > C > D
- Votant 2: B > C > D > A  
- Votant 3: C > D > A > B

Questions d'agregation:
1. Quel candidat devrait gagner selon Borda?
2. Y a-t-il un gagnant de Condorcet?
3. Que donne la regle de Copeland?

Ces questions sont au coeur de la theorie du choix social
et du theoreme d'impossibilite d'Arrow.
""")
            
    except Exception as e_gen:
        print(f"Erreur Python inattendue: {e_gen}")
        import traceback; traceback.print_exc()


--- 7.1 Ordres de Preference ---
JVM prete. Exploration du module preferences...

--- Exploration du package org.tweetyproject.preferences ---
Classes du module preferences:
   [OK] PreferenceOrder
   [--] PreferenceOrderViewer (non trouve)
   [--] Ranking (non trouve)
   [OK] ranking.Functions
   [OK] ranking.LevelingFunction
   [--] io.PreferenceParser (non trouve)
   [OK] update.Update
   [--] aggregation.BordaRule (non trouve)
   [--] aggregation.CopelandRule (non trouve)

4 classes trouvees.

--- Exemple Conceptuel: Election ---

Scenario: 3 votants, 4 candidats (A, B, C, D)

Preferences des votants:
- Votant 1: A > B > C > D
- Votant 2: B > C > D > A  
- Votant 3: C > D > A > B

Questions d'agregation:
1. Quel candidat devrait gagner selon Borda?
2. Y a-t-il un gagnant de Condorcet?
3. Que donne la regle de Copeland?

Ces questions sont au coeur de la theorie du choix social
et du theoreme d'impossibilite d'Arrow.



### 7.2 Règles d'Agrégation de Préférences

Les **règles de vote** transforment un profil de préférences individuelles en un résultat collectif. Les principales règles implémentées ou conceptualisées dans TweetyProject:

**Règles de score:**
- **Borda**: Chaque position dans un classement donne des points
- **Plurality**: Seul le premier choix compte

**Règles de Condorcet:**
- **Condorcet**: Le gagnant bat tous les autres en duel
- **Copeland**: Score = victoires en duels - défaites
- **Kemeny-Young**: Minimise la distance aux préférences individuelles

**Théorème d'Arrow:**
Aucune règle d'agrégation ne peut satisfaire simultanément: unanimité, indépendance des alternatives non pertinentes, et non-dictature (pour 3+ alternatives).

In [3]:
# --- 7.2 Règles d'Agrégation de Préférences ---
print("\n--- 7.2 Regles d'Agregation de Preferences ---")

if not jvm_ready:
    print("ERREUR: JVM non demarree.")
else:
    print("JVM prete. Demonstration des regles de vote...")
    
    # Simulation Python des règles de base (indépendant de Tweety)
    print("\n--- Simulation Python des Regles de Vote ---")
    
    # Données: préférences de 5 votants sur 4 candidats
    candidates = ['A', 'B', 'C', 'D']
    
    # Chaque liste représente le classement d'un votant (du meilleur au pire)
    preferences = [
        ['A', 'B', 'C', 'D'],  # Votant 1
        ['A', 'C', 'B', 'D'],  # Votant 2
        ['B', 'C', 'D', 'A'],  # Votant 3
        ['C', 'B', 'D', 'A'],  # Votant 4
        ['D', 'C', 'B', 'A'],  # Votant 5
    ]
    
    print(f"Candidats: {candidates}")
    print(f"Nombre de votants: {len(preferences)}")
    print("\nPreferences individuelles:")
    for i, pref in enumerate(preferences, 1):
        print(f"  Votant {i}: {' > '.join(pref)}")
    
    # Règle de Borda
    print("\n--- Regle de Borda ---")
    borda_scores = {c: 0 for c in candidates}
    n = len(candidates)
    for pref in preferences:
        for rank, candidate in enumerate(pref):
            borda_scores[candidate] += (n - 1 - rank)  # n-1 points pour le 1er, 0 pour le dernier
    
    print("Scores Borda (points):")
    for c, score in sorted(borda_scores.items(), key=lambda x: -x[1]):
        print(f"  {c}: {score} points")
    borda_winner = max(borda_scores, key=borda_scores.get)
    print(f"Gagnant Borda: {borda_winner}")
    
    # Méthode de Condorcet
    print("\n--- Methode de Condorcet ---")
    
    # Compter les victoires en duels
    def pairwise_comparison(c1, c2, preferences):
        """Compte combien de votants préfèrent c1 à c2"""
        count = 0
        for pref in preferences:
            if pref.index(c1) < pref.index(c2):
                count += 1
        return count
    
    condorcet_matrix = {}
    for c1 in candidates:
        condorcet_matrix[c1] = {}
        for c2 in candidates:
            if c1 != c2:
                condorcet_matrix[c1][c2] = pairwise_comparison(c1, c2, preferences)
    
    print("Matrice des duels (lignes battent colonnes):")
    print("     " + "  ".join(candidates))
    for c1 in candidates:
        row = [str(condorcet_matrix[c1].get(c2, '-')).center(2) for c2 in candidates]
        print(f"  {c1}  {' '.join(row)}")
    
    # Chercher le gagnant de Condorcet
    condorcet_winner = None
    for c in candidates:
        wins_all = all(condorcet_matrix[c].get(other, 0) > len(preferences)//2 
                       for other in candidates if other != c)
        if wins_all:
            condorcet_winner = c
            break
    
    if condorcet_winner:
        print(f"Gagnant de Condorcet: {condorcet_winner} (bat tous les autres en duel)")
    else:
        print("Pas de gagnant de Condorcet (cycle de Condorcet probable)")
    
    # Règle de Copeland
    print("\n--- Regle de Copeland ---")
    copeland_scores = {c: 0 for c in candidates}
    for c1 in candidates:
        for c2 in candidates:
            if c1 != c2:
                if condorcet_matrix[c1][c2] > condorcet_matrix[c2][c1]:
                    copeland_scores[c1] += 1
                elif condorcet_matrix[c1][c2] < condorcet_matrix[c2][c1]:
                    copeland_scores[c1] -= 1
    
    print("Scores Copeland (victoires - defaites):")
    for c, score in sorted(copeland_scores.items(), key=lambda x: -x[1]):
        print(f"  {c}: {score:+d}")
    copeland_winner = max(copeland_scores, key=copeland_scores.get)
    print(f"Gagnant Copeland: {copeland_winner}")


--- 7.2 Regles d'Agregation de Preferences ---
JVM prete. Demonstration des regles de vote...

--- Simulation Python des Regles de Vote ---
Candidats: ['A', 'B', 'C', 'D']
Nombre de votants: 5

Preferences individuelles:
  Votant 1: A > B > C > D
  Votant 2: A > C > B > D
  Votant 3: B > C > D > A
  Votant 4: C > B > D > A
  Votant 5: D > C > B > A

--- Regle de Borda ---
Scores Borda (points):
  C: 10 points
  B: 9 points
  A: 6 points
  D: 5 points
Gagnant Borda: C

--- Methode de Condorcet ---
Matrice des duels (lignes battent colonnes):
     A  B  C  D
  A  -  2  2  2 
  B  3  -  2  4 
  C  3  3  -  4 
  D  3  1  1  - 
Gagnant de Condorcet: C (bat tous les autres en duel)

--- Regle de Copeland ---
Scores Copeland (victoires - defaites):
  C: +3
  B: +1
  D: -1
  A: -3
Gagnant Copeland: C


### 7.3 Lien avec l'Argumentation

Les préférences sont liées à l'argumentation de plusieurs façons:

1. **Préférences sur arguments**: Ordre de préférence sur la crédibilité des arguments
2. **Argumentation sociale**: Les votes dans les SAF sont une forme d'agrégation de préférences
3. **Résolution de conflits**: Les préférences permettent de trancher entre extensions

Dans TweetyProject, le module `arg.social` (Tweety-7a) utilise des concepts de préférences pour pondérer l'acceptabilité des arguments.

In [4]:
# --- 7.3 Lien avec l'Argumentation ---
print("\n--- 7.3 Lien avec l'Argumentation ---")

if not jvm_ready:
    print("ERREUR: JVM non demarree.")
else:
    print("JVM prete. Demonstration du lien preferences-argumentation...")
    
    try:
        import jpype
        from jpype.types import *
        from org.tweetyproject.arg.dung.syntax import DungTheory, Argument, Attack
        from org.tweetyproject.arg.dung.reasoner import SimplePreferredReasoner, SimpleGroundedReasoner
        
        print("\n--- Scenario: Preferences sur Arguments ---")
        print("""
Un debat avec 3 arguments:
- A: "Le projet est rentable" (soutenu par le directeur financier)
- B: "Le projet est risque" (soutenu par l'expert technique)
- C: "Le risque est acceptable" (soutenu par le consultant)

Relations:
- A attaque B (rentabilite vs risque)
- B attaque A (risque vs rentabilite)
- C attaque B (minimise le risque)

Question: Comment les preferences des decideurs influencent le resultat?
""")
        
        # Créer le framework
        theory = DungTheory()
        a = Argument("rentable"); b = Argument("risque"); c = Argument("risque_acceptable")
        theory.add(a); theory.add(b); theory.add(c)
        theory.add(Attack(a, b)); theory.add(Attack(b, a)); theory.add(Attack(c, b))
        
        print(f"Framework: {theory}")
        
        # Calculer les extensions
        gr_reasoner = SimpleGroundedReasoner()
        pref_reasoner = SimplePreferredReasoner()
        
        grounded = gr_reasoner.getModel(theory)
        preferred = pref_reasoner.getModels(theory)
        
        print(f"\nExtension Grounded: {grounded}")
        print(f"Extensions Preferees: {preferred}")
        
        print("""
Interpretation:
- Sans preferences, l'extension grounded inclut C (risque_acceptable)
- C defend indirectement A contre B
- Si les decideurs preferent les arguments de l'expert technique (B),
  ils pourraient utiliser un framework pondere (WAF) pour ajuster les poids
- Voir Tweety-7a pour l'implementation des WAF
""")
        
    except Exception as e:
        print(f"Erreur: {e}")
        import traceback; traceback.print_exc()


--- 7.3 Lien avec l'Argumentation ---
JVM prete. Demonstration du lien preferences-argumentation...

--- Scenario: Preferences sur Arguments ---

Un debat avec 3 arguments:
- A: "Le projet est rentable" (soutenu par le directeur financier)
- B: "Le projet est risque" (soutenu par l'expert technique)
- C: "Le risque est acceptable" (soutenu par le consultant)

Relations:
- A attaque B (rentabilite vs risque)
- B attaque A (risque vs rentabilite)
- C attaque B (minimise le risque)

Question: Comment les preferences des decideurs influencent le resultat?

Framework: <{ rentable, risque, risque_acceptable },[(rentable,risque), (risque_acceptable,risque), (risque,rentable)]>

Extension Grounded: {rentable,risque_acceptable}
Extensions Preferees: [{rentable,risque_acceptable}]

Interpretation:
- Sans preferences, l'extension grounded inclut C (risque_acceptable)
- C defend indirectement A contre B
- Si les decideurs preferent les arguments de l'expert technique (B),
  ils pourraient utilise

---

## Résumé

Ce notebook a couvert:
- **Ordres de Préférence**: Représentation des préférences individuelles
- **Règles d'Agrégation**: Borda, Condorcet, Copeland
- **Théorème d'Arrow**: Limites de l'agrégation
- **Lien Argumentation**: Préférences sur arguments, WAF, SAF

## Notes sur le Module Preferences de TweetyProject

Le module `preferences` de TweetyProject est relativement moins documenté que les modules d'argumentation. Pour une utilisation avancée:
- Consultez les sources Java sur GitHub TweetyProject
- Le JAR `preferences` doit être présent dans `libs/`
- Les classes d'agrégation peuvent nécessiter une configuration spécifique

## Conclusion de la Série Tweety

Cette série de 9 notebooks a couvert les principaux modules de TweetyProject:
1. **Setup**: Configuration JVM/JPype
2. **Logiques**: PL, FOL
3. **Logiques Avancées**: DL, Modal, QBF, CL
4. **Révision**: CrMas, MUS, MaxSAT
5. **Argumentation Abstraite**: Dung, CF2
6. **Argumentation Structurée**: ASPIC+, DeLP, ABA, ASP
7a. **Frameworks Étendus**: ADF, Bipolar, WAF, SAF, SetAF, Extended
7b. **Ranking & Probabiliste**: Sémantiques de classement, loteries
8. **Dialogues**: Agents multi-agents, protocoles
9. **Préférences**: Agrégation, théorie du vote

---

**Navigation**: [← Tweety-8-Agent-Dialogues](Tweety-8-Agent-Dialogues.ipynb) | [Index](Tweety-1-Setup.ipynb)